__info__  
In cellen van dit notebook wordt regelmatig het volgende commando gebruikt: `%%time`  
Dit commando is een IPython/Jupyter notebook [magic command].  
Het commando laat zien hoelang de executie van de code in de cell duurde.  
Vaak duurt een stukje code micro-seconden (`µs`) of milli-seconden (`ms`).  

[magic command]: https://ipython.readthedocs.io/en/stable/interactive/magics.html

# Generators


## Intro

In dit notebook gaan we leren wat _generators_ zijn in Python.  

### Generator expressie

Een generator expressie is hetzelfde opgebouwd als een comprehension expressie.  
De generator schrijf je met de _parentheses_ `( )`.  

In [1]:
# generator expressie
(nummer for nummer in [1, 2, 3, ...])

<generator object <genexpr> at 0x7f6654762510>

### Wat is het verschil tussen comprehension of generators expressions

In [2]:
import sys
# import sys voor de sys.getsizeof

In [3]:
%%time

comprehension = [nummer for nummer in range(1_000_000)]
size = float(sys.getsizeof(comprehension) / (1000 ** 2))

f"{size} MB"

CPU times: user 35.9 ms, sys: 19 ms, total: 54.9 ms
Wall time: 54.7 ms


'8.448728 MB'

In [4]:
%%time

generator = (nummer for nummer in range(1_000_000))
size = float(sys.getsizeof(generator) / (1000 ** 1))

f"{size} KB"

CPU times: user 21 µs, sys: 3 µs, total: 24 µs
Wall time: 27.7 µs


'0.112 KB'

Het verschil tussen de comprehension en de generator is dat de comprehension iets met waarde heeft gemaakt.  
In dit geval: een lijst met een miljoen cijfers.  

De generator heeft nog niks gedaan, de generator geeft pas waneer de code er om vraagt.  
Dit kunnen we doen door te declareren dat de objecten in de generator in een lijst moeten komen.   

In [5]:
%%time

generator = (nummer for nummer in range(1_000_000))
list_of_generator = list(generator)
size = float(sys.getsizeof(list_of_generator) / (1000 ** 2))

f"{size} MB"

CPU times: user 34.2 ms, sys: 27.9 ms, total: 62.1 ms
Wall time: 65.7 ms


'8.448728 MB'

Er zijn alle de objecten uit de generator gehaald.  
Er kan dus gezegd worden dat de generator is uitgeput.  
De list die gemaakt is heeft dezelfde grootte als de comprehension.  

Dus wat is nou het voordeel van een generator?

In [6]:
from statistics import mean

generator = (nummer for nummer in range(1_000_000))
size_gen = (float(sys.getsizeof(item) / (1000 ** 1)) for item in generator)
mean_size = mean(size_gen)

f"Gemiddelde grootte van alle objecten in de generator: {mean_size} KB"

'Gemiddelde grootte van alle objecten in de generator: 0.027999996 KB'

Elke item die gebruikt is heeft een gemiddelde grootte van 0.03 Kilobyte.  
Dit is significant veel kleiner dan de `list` die ruim 8 Megabyte aan ruimte nodig heeft.  
Toch is dezelfde data gegenereerd wat in de `list` voorkomt.  

## Wat is een Generator

Een [generator] is een [luie] functie dat een _stream_ aan data representeerd.  
De generator genereert een [iterator] wat pas een object geeft als deze met [`next`] wordt opgevraagd.  
Als de generator is uitgeput als deze volledig is gebruikt.  
Er zal een [`StopIteration`] exception ge-_raised_ worden als er nog eens een next wordt gebruikt op de generator.    


[luie]: https://nl.wikipedia.org/wiki/Luie_evaluatie
[generator]: https://docs.python.org/3/glossary.html#term-generator
[iterator]: https://docs.python.org/3/glossary.html#term-iterator
[`next`]: https://docs.python.org/3/library/functions.html#next
[`StopIteration`]: https://docs.python.org/3/library/exceptions.html#StopIteration

In [7]:
# generator met 2 items
gen = (x for x in ['a', 'b'])

print("next nr. 1: ", next(gen))
print("next nr. 2: ", next(gen))
# print("next nr. 3: ", next(gen))  # uncomment voor StopIteration exception

next nr. 1:  a
next nr. 2:  b


### Generator functie met `yield`

Een generator is dus een functie, maar tot nu toe is alleen de [generator expressie] geschreven.  
In een functie kan de [`yield`] statement gebruikt worden om van een functie een generator te maken.  

[`yield`]: https://docs.python.org/3/reference/expressions.html#yield-expressions
[generator expressie]: https://docs.python.org/3/reference/expressions.html#generator-expressions

In [8]:
def yield_int(nummer: int):
    yield int(nummer)
    yield int(nummer) + 1

In [9]:
gen = yield_int(1)
gen

<generator object yield_int at 0x7f662671d350>

In [10]:
print(next(gen))
print(next(gen))
# next(gen)  # uncomment voor StopIteration exception

1
2


Een generator functie heeft in de body de `yield` statement.  
`yield` zorgt ervoor dat de functie halverwegen stopt.  
Het is alsof de functie meerdere `return` statements heeft.  

Het is ook mogenlijk om `yield` in een loop te gebruiken.  
Hieronder een voorbeeld van een generator dat eindeloos calculaties maakt.  

In [11]:
def endless_loop(start: int, multiplier: int):
    while True:
        output = start * multiplier
        yield output
        start = output

In [12]:
endless_gen = endless_loop(1, 2)

for loop_num, nummer in enumerate(endless_gen, start=1):
    print(f"{loop_num}: {nummer}")
    if loop_num >= 10:
        break

1: 2
2: 4
3: 8
4: 16
5: 32
6: 64
7: 128
8: 256
9: 512
10: 1024


En ook na deze for-loop wacht de generator totdat er een nieuw nummer wordt opgevraagd.

In [13]:
# loop nummer 11
next(endless_gen)

2048

De generator kan ook vroegtijdig gestopt worden met het `.close()` attribute.  
Als hierna nogmaals hetvolgende object wordt opgevraagd zal de generator een `StopIteration` exception geven.

In [14]:
endless_gen.close()

In [15]:
# next(endless_gen)  # uncomment voor StopIteration exception

### Twee-weg communitatie met `yield`

`yield` geeft aan dat de functie halverwegen stopt en iets teruggeeft.  
Maar `yield` kan ook data ontvangen en dit kan weer in de generator gebruiken.

In [16]:
def intermittent_sum():
    outgoing = 0
    while True:
        incomming = yield outgoing
        outgoing = incomming + outgoing

Voordat je iets kan verzenden naar een generator moet deze eerst bij een `yield` statement wachten.

In [17]:
isum = intermittent_sum()
# num = isum.send(None)
num = next(isum)  # start de generator (en yield geeft iets terug)
num

0

In [18]:
num = isum.send(2)  # stuur een waarde op naar de generator en ontvang een waarde
num

2

De generator functie is een luie functie.  
Het wacht todat het iets gevraagd wordt.  
Pas bij de eerste keer dat er iets gevraagd wordt zal de generator naar de `yield` gaan.  
Hier geeft de generator terug wat er is opgevraagd en wacht totdat het weer wordt opgeroepen om iets te doen.  

Het is niet mogenlijk om een generator een __niet__ `None` waarde op te sturen als deze net gestart is.  
`next(generator)` is hetzelfde als `generator.send(None)`  
Pas hierna is de generator gereed om iets te ontvangen wat met `generator.send(...)` naar de generator wordt verstuurd.


In [19]:
from IPython import display
display.HTML(filename=r'./docs/yield_generator_send.html')

In [20]:
# intermittent_sum().send(1)  # uncomment voor TypeError exception

### Coroutines

Met Generators kan je coroutines maken.  
`yield` wacht op inkomende data of een signaal om weer aan de slag te gaan.  
Omdat `yield` sommige delen van het programma laat wachten, kan het programma ondertussen met iets anders bezig zijn.  
Dit is de [basis] van [asynchroon] programmeren en Python heeft daar de [asyncio] library voor.


Hieronder een voorbeeld van een coroutine constructie met Generators

[basis]: https://docs.python.org/3/library/asyncio-task.html#generator-based-coroutines
[asynchroon]: https://en.wikipedia.org/wiki/Asynchronous_I/O
[asyncio]: https://docs.python.org/3/library/asyncio.html

In [21]:
import os
import sys
import time


def reader(filepath, generator):
    with open(filepath, 'r') as text_file:
        text_file.seek(0, os.SEEK_END)  # plaats de cursor op het einde van de file.
        while True:
            try:
                time.sleep(0.1)
                data = text_file.read()
                if data:
                    generator.send(data)  # stuur de data naar de gegeven generator
            except KeyboardInterrupt:
                break


def switcher(generators_list_tup: list[tuple]):
    """[(Generator, "text to search"), ... ]"""
    while True:
        data = yield
        for gen, text in generators_list_tup:
            if text in str(data):  # zoek in text
                gen.send(data)


def apply(func, generator):
    while True:
        data = yield
        output = func(data)  # gebruik de function met de data
        generator.send(output)


def print_out():
    while True:
        data = yield  # wacht op de inkomende data
        print(data, end='')

In [22]:
file = "data_stream.txt"
open(file, 'w').close()  # creer de file

printer = print_out()
next(printer)

upper = apply(str.upper, printer)
next(upper)

lower = apply(str.lower, printer)
next(lower)

generator_list_tup = [(lower, "debug"), (upper, "error")]
switch = switcher(generator_list_tup)
next(switch)

reader(file, switch)  # schrijf error of debug in de data_stream.txt file aan en save het.

De code van hierboven in een diagram.

In [23]:
from IPython import display
display.HTML(filename=r'./docs/coroutineflow.html')

### `yield from` syntax sugar

Het keyword `from` is al bekend bij het importeren van packages of modules.  
Maar het is ook bruikbaar achter een `yield` statement.  
Het is een kortere manier om een for-loop met een `yield` er in te schrijven.  

In [24]:
def range_0_10():
    for i in range(0, (10 + 1)):
        yield i
        
def range_10_20():
    yield from range(10, (20 + 1))

def create_iter(serie):
    yield from serie

In [25]:
for i in range_0_10():
    print(i, end=' ')

0 1 2 3 4 5 6 7 8 9 10 

In [26]:
for i in range_10_20():
    print(i, end=' ')

10 11 12 13 14 15 16 17 18 19 20 

In [27]:
gen = range_0_10()
for i in create_iter(gen):
    print(i, end=' ')

0 1 2 3 4 5 6 7 8 9 10 

Het is mooie syntax voor een generator wat eindeloos over een bepaalde serie loopt.

In [28]:
def cycle(serie):
    while True:
        yield from serie
        
gen = cycle("cycle")
for loop_num, item in enumerate(gen, start=1):
    print(item, end=' ')
    if loop_num >= 10:
        break

c y c l e c y c l e 

### Mix van `yield` en `return`

Het is toegestaan om `yield` en `return` in een functie te hebben.  
Maar een generator geeft een generator terug, dus kan je afvragen wat gebeurd met de `return`.  

De waarde wat met de `return` wordt terug gegeven, komt terug in de `.value` attribute van de `StopIteration` error.  

In [29]:
def gen_return():
    yield 'nog nooit gezien'
    return 'in productie code'

In [30]:
gen = gen_return()
letter = next(gen)
letter

'nog nooit gezien'

In [31]:
try:
    next(gen)
except StopIteration as ret:
    letter = ret.value  # return value in de StopIteration exception

letter

'in productie code'